# Import Training Data, Add Labels, Check

In [1]:
#add labels, print dataframes to check
import pandas as pd
import numpy as np
train_map = pd.read_csv('train.map', header = None, delimiter=' ', names= ['labels', 'label_id'])
train_data = pd.read_csv('train.data', header = None, delimiter=' ', names=['doc_id', 'word_id', 'count'] )
train_label = pd.read_csv('train.label', header = None, delimiter=' ', names= ['label_id'])
train_label=train_label.reset_index()
train_label.columns = ['doc_id', 'label_id']
#add one to the doc ID so that there are no zeroes
train_label['doc_id']=train_label['doc_id']+1
print train_map[:10]
print train_data[:10]
train_label

                     labels  label_id
0               alt.atheism         1
1             comp.graphics         2
2   comp.os.ms-windows.misc         3
3  comp.sys.ibm.pc.hardware         4
4     comp.sys.mac.hardware         5
5            comp.windows.x         6
6              misc.forsale         7
7                 rec.autos         8
8           rec.motorcycles         9
9        rec.sport.baseball        10
   doc_id  word_id  count
0       1        1      4
1       1        2      2
2       1        3     10
3       1        4      4
4       1        5      2
5       1        6      1
6       1        7      1
7       1        8      1
8       1        9      3
9       1       10      9


,doc_id,label_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,10,1


# Merge Training Data Tables

# Get Total Doc Count & Docs Per Class

In [2]:
#Get total document count by searching the unique values, this could also be done by taking max of the ID 
total_doc_count= train_data_joined['doc_id'].unique().shape

# Count number of docs for each class
docs_per_class=train_data_joined.groupby('label_id').doc_id.nunique()

NameError: name 'train_data_joined' is not defined

# Calculate Pi for Classes

In [ ]:
# Calculate Pi for Each Class, this is the number of documents in a class/total number of docs
df_docs_frac_per_class=np.log(train_data_joined.groupby('label_id').doc_id.nunique()/total_doc_count)
pi_values=df_docs_frac_per_class.reset_index()
pi_values.columns=['label_id','log_pi']
#pi_values

# Calculate Total Words Per Label

In [ ]:
words_per_class=train_data_joined[['label_id','count']].groupby('label_id').sum().reset_index()
words_per_class.columns=['label_id','words_per_label']
#words_per_class

# Calculate Total Words Per Doc

In [ ]:
sum_words_per_doc=train_data_joined.groupby(['label_id','word_id'])['count'].sum().reset_index()
sum_words_per_doc.head(5)

# Set Up Training Data Probability Distribution (Pj)

# SACHIN: THIS IS WHERE I'M HAVING ISSUES, I NEED TO THE PANDAS DATAFRAME UNSTACKED AND THE REST SHOULD WORK 

In [ ]:
len(train_label)

In [ ]:
#count of a specific word in the same class/total number of words in each class
#Alpha value for smoothing
a = 0.001

#Calculate probability of each word based on the class
pb_j=train_data_joined.groupby('label_id')
pb_ij=train_data_joined[['label_id','word_id','count']].groupby(['label_id','word_id'])
prob =  (pb_ij['count'].sum() + 1) / (pb_j['count'].sum() + 61188) 
k=prob.unstack()
k2=k.fillna(1.0/61188)
k3=k2.stack()
prob_distr=k3.reset_index()
prob_distr.columns=['label_id','word_id','log(prob)']
prob_distr


In [ ]:
# #add one for all of the null values as (1/count+V+1)
# for c in range(1,21):
#     prob_distr.loc[c,:] = prob_distr.loc[c,:].fillna(a/(pb_j['count'].sum()[c] + 16689))
    
# #Convert to dictionary for greater speed
# prob_distr_dict = prob_distr.to_dict()

# #prob_distr=prob_distr.reset_index()
# x=prob_distr.unstack
# x
# ###### I NEED X AS IT IS HERE Reindexed, But I cant save the instance Method

# Import Test Data

In [ ]:
test_map = pd.read_csv('test.map', header = None, delimiter=' ', names= ['test_labels', 'test_label_id'])
test_data = pd.read_csv('test.data', header = None, delimiter=' ', names=['test_doc_id', 'test_word_id', 'test_count'] )
test_label = pd.read_csv('test.label', header = None, delimiter=' ', names= ['test_label_id'])
test_label=test_label.reset_index()
test_label.columns = ['test_doc_id', 'label_id']
print test_map.head(5)
print test_data.head(5)
test_label.head(5)

# Merge Test Data

In [ ]:
#test_label['test_doc_id']=test_label['test_doc_id']
merged_test_data = pd.merge( test_label, test_data, on='test_doc_id', how="inner")
merged_test_data.head(5)

# Probability of Training Data

# Get Single Test Document for Testing

In [ ]:
df=merged_test_data.loc[merged_test_data['test_doc_id']==1];
df.columns = df.columns.str.replace('test_word_id','word_id')
#sum_words_per_doc=df.groupby(['word_id','doc_id'])['count'].sum().reset_index()
sum_words_per_doc.head(5)
df.head(5)

# Setup Bayesian Algorithm: Map Probability for a Doc that Belongs to Each Class
#1) Match on Doc ID, #2 Pull Out Label ID, Probability Word Belongs to that class, and count of word occurence in document; sort by word ID so you can see the map of each word mapping to each class #3 multiply words to get prior probability #4 sum prior probability over each class (it can be added due to the log). #5 raname so that ID's line up and pie values for each class can be multiplied #6 add up the pi with the prior prob, can add because of the log values #7 Answer gives the probability that the document belongs to each class
#Log(Probability(word/class)^N,words)summed

In [ ]:
single_doc_dist= pd.merge(prob_distr,df, on='word_id')
single_doc_dist= single_doc_dist[['label_id_x','word_id','log(prob)','test_count']].sort_values(['word_id','label_id_x'])
single_doc_dist ['prior_prob']= single_doc_dist['test_count']*single_doc_dist['log(prob)']
single_doc_dist=single_doc_dist.groupby(['label_id_x'])['prior_prob'].sum().round(3).reset_index()
single_doc_dist.columns=['label_id','prior_prob']
single_doc_dist = pd.merge (single_doc_dist,pi_values, on='label_id' )
single_doc_dist['probability']=single_doc_dist['prior_prob']+single_doc_dist['log_pi']
single_doc_dist


In [ ]:
single_doc_dist['probability'].idxmax()+1

# Setup Function to Classify One Document

In [ ]:
def SingleDoc_NaiveBayesClassifier(doc):
    df=merged_test_data.loc[merged_test_data['test_doc_id']==doc];
    df.columns = df.columns.str.replace('test_word_id','word_id')
    ###### Copy below this line, above is for the function
    single_doc_dist= pd.merge(prob_distr,df, on='word_id')
    single_doc_dist= single_doc_dist[['label_id_x','word_id','log(prob)','test_count']].sort_values(['word_id','label_id_x'])
    single_doc_dist ['prior_prob']= single_doc_dist['test_count']*single_doc_dist['log(prob)']
    single_doc_dist=single_doc_dist.groupby(['label_id_x'])['prior_prob'].sum().round(3).reset_index()
    single_doc_dist.columns=['label_id','prior_prob']
    single_doc_dist = pd.merge (single_doc_dist,pi_values, on='label_id' )
    single_doc_dist['probability']=single_doc_dist['prior_prob']+single_doc_dist['log_pi']
    class_rating = single_doc_dist['probability'].idxmax()+1
    print class_rating
SingleDoc_NaiveBayesClassifier(2)

# Setup New Classifier Function for All Words

# Sachin, I would like to finish this tomorrow, I cant find the error that is 
# giving 100% accuracy.

In [ ]:
def NaiveBayesClassifier(df):
    df_all= df
    df_all.columns = df_all.columns.str.replace('test_word_id','word_id')
    all_doc_dist= pd.merge(prob_distr,df_all, on=['label_id','word_id'])
    all_doc_dist= all_doc_dist[['label_id','word_id','log(prob)','test_count','test_doc_id']].sort_values(['word_id','label_id'])
    all_doc_dist ['prior_prob']= all_doc_dist['test_count']*all_doc_dist['log(prob)']
    all_doc_dist=all_doc_dist.groupby(['test_doc_id','label_id'])['prior_prob'].sum().reset_index()
    all_doc_dist.columns=['test_doc_id','label_id','prior_prob']
    all_doc_dist = pd.merge (all_doc_dist,pi_values, on='label_id' )
    all_doc_dist['probability']=all_doc_dist['prior_prob']+all_doc_dist['log_pi']
    classified_dataframe= all_doc_dist.sort_values('probability', ascending=False).groupby('test_doc_id', as_index=False).first()
    classified_dataframe= classified_dataframe[['test_doc_id','label_id']]
    return classified_dataframe
classified_dataframe=NaiveBayesClassifier(merged_test_data)
# print classified_dataframe
classified_dataframe

# Test Accuracy of Classifier

In [ ]:
test_label.columns = ['doc_id', 'test_label_id']
classified_dataframe.columns = ['doc_id', 'predicted_label_id']
guess_to_real= pd.merge(classified_dataframe,test_label, on='doc_id')

In [ ]:
guess_to_real['err']=guess_to_real.apply(lambda row:row['predicted_label_id']-row['test_label_id'], axis=1)

In [ ]:
np.unique(guess_to_real['err'])

In [ ]:
vocab_list = pd.read_csv('vocabulary.txt', header = None, delimiter=' ', names= ['vocab'])
vocab_list

In [ ]:
#Vocab=61188
#Vocab*20=1223760
#